In [ ]:
import os
import json
from typing import List
import math
from glob import glob

from tqdm import tqdm
import cv2
import numpy as np
from PIL import Image
from pandas import json_normalize
import pandas as pd
import re
import warnings

import albumentations as A

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

# !pip freeze | grep albumentations
# !pip install albumentations==0.4.6
#from albumentations.pytorch.transforms import ToTensorV2

from matplotlib import pyplot as plt
warnings.filterwarnings(action='ignore')


rootfolder = '/content/drive/My Drive/dacon - 위성 객체 인식/위성객체폴더'
DIR_TRAIN = os.path.join(rootfolder,'train/images')


In [ ]:
json_ls = os.listdir(os.path.join(rootfolder,'train/json/'))

In [ ]:
nm = 'OBJ03589_PS3_K3A_NIA0151.json'

In [ ]:
with open(os.path.join(rootfolder,f'train/json/{nm}')) as json_open:
    json_file = json.load(json_open)

In [ ]:
json_file

{'features': [{'geometry': {'coordinates': [[[1, 1, 0.0],
      [1, 1, 0.0],
      [1, 1, 0.0],
      [1, 1, 0.0]]],
    'type': 'Polygon'},
   'properties': {'building_imcoords': 'EMPTY',
    'image_id': 'OBJ03589_PS3_K3A_NIA0151.png',
    'ingest_time': '2020-09-18T08:50:32.091450Z',
    'object_angle': 6.16525663383006,
    'object_imcoords': '480.7764823560916,476.8751436078733,548.6785206340824,484.9200695789738,539.6848170639626,560.8301284499863,471.7827787859718,552.7852024788859',
    'road_imcoords': 'EMPTY',
    'type_id': '3',
    'type_name': 'civilian aircraft'},
   'type': 'Feature'},
  {'geometry': {'coordinates': [[[1, 1, 0.0],
      [1, 1, 0.0],
      [1, 1, 0.0],
      [1, 1, 0.0]]],
    'type': 'Polygon'},
   'properties': {'building_imcoords': 'EMPTY',
    'image_id': 'OBJ03589_PS3_K3A_NIA0151.png',
    'ingest_time': '2020-09-18T08:50:32.091450Z',
    'object_angle': 0.05009021927389256,
    'object_imcoords': '175.0753841081058,928.0478765667175,183.1000964147065

In [ ]:
data_consist = pd.DataFrame()
nm = []
bbox_ls = []
for idx, json_nm in enumerate(json_ls):
  with open(os.path.join(rootfolder,f'train/json/{json_nm}')) as json_open:
    json_file = json.load(json_open)

  nm.append(json_nm.split('.')[0])
  bbox_ls.append(len(json_file['features']))

In [ ]:
json_file

In [ ]:
img_files = data_consist['nm'].iloc[:3].tolist()

In [ ]:
img_files

['OBJ04712_PS3_K3A_NIA0313',
 'OBJ03922_PS3_K3A_NIA0165',
 'OBJ03977_PS3_K3A_NIA0166']

In [ ]:
img_id_map= {img_file:i+1 for i, img_file in enumerate(list(set(img_files)))}
image_ids = [img_id_map[img_file] for img_file in img_files]

In [ ]:
img_id_map

{'OBJ03922_PS3_K3A_NIA0165': 1,
 'OBJ03977_PS3_K3A_NIA0166': 3,
 'OBJ04712_PS3_K3A_NIA0313': 2}

In [ ]:
image_ids

[2, 1, 3]

In [ ]:
for imgfile in tqdm(img_id_map):
  img_file

  0%|          | 0/3 [00:00<?, ?it/s]


NameError: ignored

In [ ]:
data_consist['nm'] = nm
data_consist['bbox_len'] = bbox_ls

In [ ]:
data_consist.sort_values('bbox_len',ascending = False)

,nm,bbox_len
460,OBJ03758_PS3_K3A_NIA0157,1637
461,OBJ04053_PS3_K3A_NIA0170,1411
540,OBJ04096_PS3_K3A_NIA0174,1170
483,OBJ03789_PS3_K3A_NIA0158,1160
526,OBJ05425_PS3_K3A_NIA0349,1109
...,...,...
425,OBJ04620_PS3_K3A_NIA0303,1
394,OBJ02596_PS3_K3_NIA0124,1
422,OBJ04134_PS3_K3A_NIA0177,1
416,OBJ02310_PS3_K3_NIA0118,1


In [ ]:
img = cv2.imread(os.path.join(rootfolder,'train/images/OBJ03589_PS3_K3A_NIA0151.png') , cv2.COLOR_BGR2RGB)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
#plt.figure(figsize = (16,16)) #용량 문제로 주석 처리
plt.imshow(img) 